# Interact with MindSphere APIs
## Predictive Learning - Getting Started Code Snippets

Short overview on how to authenticate inside Predictive Learning against MindSphere Services (APIs) and use them in your notebook.  

### How it works
- Predictive Learning provices base URL in an environment variable. This can be accessed via ```os.environ["GATEWAY_ENDPOINT"]```
- In the Service URL, ```api``` has to be replaced with ```gateway```.  
  For Asset-Management Service, the base URL then e.g. changes as follows:  
  | Base URL Asset Management [general] | Base URL Asset Management [in PRL] |
  | --- | --- | 
  | ```/api/assetmanagement/v3```  | ```/gateway/assetmanagement/v3``` |
  
  the full request then goes against  
  ```https://prl-internal-gateway-prod.analytics.eu-central.internal.mindsphere.io:8443/gateway/assetmanagement/v3/<Service>```  
  Likewise this pattern needs to be followed for any other API endpoint. 

### Resources
- [MindSphere Analytical APIs](https://developer.mindsphere.io/apis/index.html#analytics-services) 


## Setup (install libraries, config endpoints)

In [ ]:
!python3 -m pip install requests urllib --user

In [ ]:
#init services
import os
import requests
import urllib.parse
# import base64
# from pathlib import Path
# import pyarrow as pa
# import io
import json

service_urls = {
    "base": os.environ["GATEWAY_ENDPOINT"] + "/gateway",
    "AM-services": "/assetmanagement/v3",
    "IDL-services": "/datalake/v3",
    "IDL-generateUploadObjectUrls": "/generateUploadObjectUrls",
    "IDL-generateDownloadObjectUrls": "/generateDownloadObjectUrls",
    "IDL-objects": "/objects",
    "AM-assets": "/assets"
}

# headers for idl requests
headers_idl = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Content-Type": "application/json",
}

## 1 - Asset List

In [ ]:
#example to show asset-list for tenant

# specifiy service URL
ServiceUrl = f"{service_urls['base']}{service_urls['AM-services']}{service_urls['AM-assets']}?&size=100"

response = requests.get(ServiceUrl)

response_data = json.loads(response.text)

#check and print response
if response.status_code == 200:
    if not response_data.get("_embedded"): 
        print ('[200] - OK, but could not read JSON response propperly (error!)')
    else:
        print ('[200] - OK, root level asset list retrived')
        #save response for further processing
        rootAssetList = response_data.get("_embedded").get("assets")
               
        print("Asset-Overview:")
        print("{0:40} {1:40} {2:30}".format("Name", "Asset-ID", "Type"))
        print("="*100)

        for asset in rootAssetList:  
            currentAssetId = asset["assetId"]
            print("{0:40} {1:40} {2:30}".format(asset.get("name"), asset.get("assetId"), asset.get("typeId")))
                
else:
    print('Error querying asset list:')
    print(response.status_code)
    pprint.pprint(response_data)

## 1 - IDL files

In [ ]:
#idl_path = "path/to/folder"
# note: this path DOES NOT start with 'data/ten=<youtTenant>'
idl_path = "/"  #e.g. "/myData/myProject/machineData1"
        
# specifiy service URL and body
ServiceUrl = f"{service_urls['base']}{service_urls['IDL-services']}{service_urls['IDL-objects']}"
body = {"path": idl_path} 

# GET request
response = requests.get(ServiceUrl, data)

# loop through response
objs = [
    obj["path"].split("ten=")[-1].split("/", maxsplit=1)[-1]
    for obj in response.json()["objects"]
]

# Only 1000 objects are returned for each request. Using page.nextToken
while "page" in r_obj:
    # update params with page.nextToken for the next request
    params["pageToken"] = r.json()["page"]["nextToken"]
    r_obj = requets.get(
        f"{urls['base']}/{urls['objects']}", params=params, headers=headers_idl
    )
    objs += [
        obj["path"].split("ten=")[-1].split("/", maxsplit=1)[-1]
        for obj in r_obj.json()["objects"]
    ]

response_data = json.loads(r_obj.text)
print(response_data)